In [7]:
%pip install pypdf2

     |████████████████████████████████| 232 kB 1.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install pdfminer

     |████████████████████████████████| 4.2 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 59.9 MB/s eta 0:00:01
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140127 sha256=633fe88099e758c3b62d33bcf6bdd962478daf6d23c2287d1e40a5f5c429e82c
  Stored in directory: /Users/a.arkhipova/Library/Caches/pip/wheels/1c/28/7d/f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install pdfminer.six

     |████████████████████████████████| 5.6 MB 877 kB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 106.0 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 83.9 MB/s eta 0:00:01
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.7
    Uninstalling cryptography-3.4.7:
      Successfully uninstalled cryptography-3.4.7
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pdfplumber

     |████████████████████████████████| 59 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 60.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 64.7 MB/s eta 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.2.0
    Uninstalling Pillow-8.2.0:
      Successfully uninstalled Pillow-8.2.0
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install pdf2image

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install pytesseract

     |████████████████████████████████| 65 kB 775 kB/s eta 0:00:01
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-core 0.4.0 requires jupyter-server<3,>=1.21, but you have jupyter-server 1.4.1 which is incompatible.
aext-assistant 0.4.0 requires jupyter-server<3,>=1.21, but you have jupyter-server 1.4.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import string
from PyPDF2 import PdfReader

import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
import pdfplumber
from PIL import Image
from pdf2image import convert_from_path
import pytesseract 
import os

In [18]:
reader = PdfReader("Гражданский кодекс Российской Федерации (часть первая).pdf")
text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

In [20]:
pdf_path = "Гражданский кодекс Российской Федерации (часть первая).pdf"

pdfFileObj = open(pdf_path, 'rb')
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

text_per_page = {}

In [22]:
def text_extraction(element):
    line_text = element.get_text()
    
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            for character in text_line:
                if isinstance(character, LTChar):
                    line_formats.append(character.fontname)
                    line_formats.append(character.size)
    format_per_line = list(set(line_formats))
    
    return (line_text, format_per_line)

def crop_image(element, pageObj):
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

def image_to_text(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def extract_table(pdf_path, page_num, table_num):
    pdf = pdfplumber.open(pdf_path)
    table_page = pdf.pages[page_num]
    table = table_page.extract_tables()[table_num]
    return table

def table_converter(table):
    table_string = ''
    for row_num in range(len(table)):
        row = table[row_num]
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    table_string = table_string[:-1]
    return table_string

In [25]:
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    table_num = 0
    first_element= True
    table_extraction_flag= False
    pdf = pdfplumber.open(pdf_path)
    page_tables = pdf.pages[pagenum]
    tables = page_tables.find_tables()


    page_elements = [(element.y1, element) for element in page._objs]
    page_elements.sort(key=lambda a: a[0], reverse=True)

    for i,component in enumerate(page_elements):
        pos= component[0]
        element = component[1]
        
        if isinstance(element, LTTextContainer):
            if table_extraction_flag == False:
                (line_text, format_per_line) = text_extraction(element)
                page_text.append(line_text)
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                pass

        if isinstance(element, LTFigure):
            crop_image(element, pageObj)
            convert_to_images('cropped_image.pdf')
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            page_text.append('image')
            line_format.append('image')

    dctkey = 'Page_'+str(pagenum)
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

pdfFileObj.close()

result = ''.join(text_per_page['Page_0'][4])
print(result)

17.02.2025, 16:01
"Гражданский кодекс Российской Федерации (часть первая)" от 30.11.1994 N 51-ФЗ (ред. от 08... \ КонсультантПлюс
 
30 ноября 1994 года 
N 51-ФЗ
 
 
ГРАЖДАНСКИЙ КОДЕКС РОССИЙСКОЙ ФЕДЕРАЦИИ
 
ЧАСТЬ ПЕРВАЯ
 
Принят
Государственной Думой
21 октября 1994 года
Часть вторая, часть третья и часть четвертая
Гражданского кодекса РФ
введены в информационный банк отдельными документами
Список изменяющих документов
(в ред. Федеральных законов от 20.02.1996 N 18-ФЗ, от 12.08.1996 N 111-ФЗ,
от 08.07.1999 N 138-ФЗ, от 16.04.2001 N 45-ФЗ, от 15.05.2001 N 54-ФЗ,
от 21.03.2002 N 31-ФЗ, от 14.11.2002 N 161-ФЗ, от 26.11.2002 N 152-ФЗ,
от 10.01.2003 N 15-ФЗ, от 23.12.2003 N 182-ФЗ, от 29.06.2004 N 58-ФЗ,
от 29.07.2004 N 97-ФЗ, от 29.12.2004 N 192-ФЗ, от 30.12.2004 N 213-ФЗ,
от 30.12.2004 N 217-ФЗ, от 02.07.2005 N 83-ФЗ, от 21.07.2005 N 109-ФЗ,
от 03.01.2006 N 6-ФЗ, от 10.01.2006 N 18-ФЗ, от 03.06.2006 N 73-ФЗ,
от 30.06.2006 N 93-ФЗ, от 27.07.2006 N 138-ФЗ, от 03.11.2006 N 175-ФЗ,
от 04.12.2

In [34]:
result = ''.join(text_per_page['Page_3'][4])
print(result)

17.02.2025, 16:01
"Гражданский кодекс Российской Федерации (часть первая)" от 30.11.1994 N 51-ФЗ (ред. от 08... \ КонсультантПлюс
2. Гражданское законодательство состоит из настоящего Кодекса и принятых в соответствии с ним
иных  федеральных  законов  (далее  -  законы),  регулирующих  отношения,  указанные  в  пунктах  1  и  2
статьи 2 настоящего Кодекса.
Нормы гражданского права, содержащиеся в других законах, должны соответствовать настоящему
Кодексу.
2.1.  Внесение  изменений  в  настоящий  Кодекс,  а  также  приостановление  действия  или  признание
утратившими  силу  положений  настоящего  Кодекса  осуществляется  отдельными  законами.  Положения,
предусматривающие  внесение  изменений  в  настоящий  Кодекс,  приостановление  действия  или
признание  утратившими  силу  положений  настоящего  Кодекса,  не  могут  быть  включены  в  тексты
законов,  изменяющих  (приостанавливающих  действие  или  признающих  утратившими  силу)  другие
законодательные  акты  Российской  Федерации  и

In [3]:
def extract_text_from_pdf(pdf_path):
    """Извлекает текст из PDF-документа."""
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n" 
    return text

def process_text(text):
    """Обрабатывает текст, удаляя ссылки, колонтитулы, сноски, названия статей и объединяя перечисления."""
    dataset = []
    
    ignored_patterns = [
        r'\d{2}\.\d{2}\.\d{4}', 
        r'КонсультантПлюс',     
        r'Позиции высших судов', 
        r'Гражданский кодекс',   
        r'ред\. от',            
        r'КонсультантПлюс: примечание',  
        r'Статья\s+\d+\.',       
        r'Глава\s+\d+\.',        
        r'Подраздел\s+\d+\.',     
        r'Раздел\s+\d+\.',       
        r'https?://\S+',   
        r'N\s+\d+-П\.',          
    ]
    
    text = re.sub(r'[А-ЯЁ\s]+\n', '', text)
    text = re.sub(r'\([^()]*\)', '', text)
    lines = text.splitlines()
    
    current_paragraph = ""
    for line in lines:
        if any(re.search(pattern, line) for pattern in ignored_patterns):
            continue
        
        line = re.sub(r'\([^()]*\)', '', line).strip()
        
        if re.match(r'^\s*\d+\.', line) or re.match(r'^\s*\d+\)', line) or re.match(r'^\s*\d+\.\d+\)', line):
            if current_paragraph:  
                dataset.append(current_paragraph.strip())
                current_paragraph = ""
            
            line = re.sub(r'^\s*\d+\.\s*', '', line) 
            line = re.sub(r'^\s*\d+\)\s*', '', line) 
            line = re.sub(r'^\s*\d+\.\d+\)\s*', '', line) 
        
        if ":" in line and re.search(r'\d+\)', line):
            current_paragraph += " " + line.strip()
        else:
            current_paragraph += " " + line.strip()
    
    if current_paragraph:
        dataset.append(current_paragraph.strip())
    
    return dataset

def save_dataset(dataset, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for line in dataset:
            f.write(line + '\n')
            
def save_to_excel(dataset, output_file):
    df = pd.DataFrame(dataset, columns=["Text"]) 
    df.to_excel(output_file, index=False)  

In [62]:
pdf_path = "Гражданский кодекс Российской Федерации (часть первая).pdf"  
output_file = 'dataset.txt'  

text = extract_text_from_pdf(pdf_path)
dataset = process_text(text)
#save_dataset(dataset, output_file)
save_to_excel(df, output_file)

print(f"Датасет сохранен в файл: {output_file}")

<ipython-input-61-242cb9c8be89>:79: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel('Гражданский кодекс (часть первая).xlsx', index=False)  # Сохраняем в Excel без индексов


Датасет сохранен в файл: dataset.txt


In [7]:
pdf_path = "Гражданский кодекс Российской Федерации (часть вторая).pdf" 
output_file = 'Гражданский кодекс Российской Федерации (часть вторая).xlsx' 

text2 = extract_text_from_pdf(pdf_path)
dataset2 = process_text(text2)
#save_dataset(dataset2, output_file)
save_to_excel(dataset2, output_file)

print(f"Датасет сохранен в файл: {output_file}")

<ipython-input-3-391e920b8186>:79: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_file, index=False)  # Сохраняем в Excel без индексов


Датасет сохранен в файл: Гражданский кодекс Российской Федерации (часть вторая).xlsx
